# Scrape Data From Reddit Pushshift

In [8]:
from Reddit_Loader import Loader
import datetime
import pandas as pd
import pickle


In [2]:
# Set Start and End Date
START = "01/09/2021"
END = datetime.datetime.now().strftime("%d/%m/%Y") #today

In [3]:
# Load data from Pushshift and store it as a dataframe
l = Loader("gme", START, END, "wallstreetbets")
submissions = l.load_submissions()

Response cache key: 78b74e1becebdb8c4b06175744a58ef7
Retrieved 694 submissions from Pushshift.


In [7]:
# Save as csv
submissions.to_csv('./data/submissions.csv', encoding='utf-8', index=False)

In [9]:
# Save dataframe as pickle
submissions.to_pickle("./data/submissions.pkl")

In [10]:
print(pd.read_pickle("./data/submissions.pkl"))

    all_awardings  allow_live_comments                author  \
0              []                False               didiamc   
1              []                False     Papo_of_the_papos   
2              []                False          Shethjugal03   
3              []                False               1xSoSx1   
4              []                False                aznkor   
..            ...                  ...                   ...   
689            []                False      BigWhaleOfficial   
690            []                False  Affectionate-Duck452   
691            []                False               dvinz01   
692            []                False      BigWhaleOfficial   
693            []                False     BurgundySerpent72   

    author_flair_css_class author_flair_richtext author_flair_text  \
0                     None                    []              None   
1                     None                    []              None   
2                    